## a JSON format compatible with the Pandas data structure 
-------     
      
### introduction
The data type is not explicitely taken into account in the current JSON interface.     
    
The existing solution is to use a data schema in addition.

### proposal
To have a simple, compact and reversible solution, I propose to use the [JSON-NTV format (Named and Typed Value)](https://github.com/loco-philippe/NTV#readme) - which integrates the notion of type - and its JSON-TAB variation for tabular data.    
This solution allows to include a large number of types (not necessarily Pandas dtype).

### content
This NoteBook uses examples to present some key points

*(active link on jupyter Notebook or Nbviewer)*
- [Current Json interface](#Current-Json-interface)
    - [Example : simple column](#Example-:-simple-column)
    - [Types and Json interface](#Types-and-Json-interface)
    - [Data compactness](#data-compactness)
    - [External types](#external-types)
- [Series](#Series)
    - [Simple example](#Simple-example)
    - [Typed example](#Typed-example)
    - [Examples with a non-Pandas type](#Examples-with-a-non-Pandas-type)
    - [Categorical examples](#Categorical-examples)
- [DataFrame](#DataFrame)
    - [Initial example](#Initial-example)
    - [Complete example](#Complete-example)
    - [Json data can be annotated](#Json-data-can-be-annotated)
    - [Categorical data can be included](#Categorical-data-can-be-included)
    
### References
- [JSON-NTV specification](https://github.com/loco-philippe/NTV/blob/main/documentation/JSON-NTV-standard.pdf)
- [JSON-TAB specification](https://github.com/loco-philippe/NTV/blob/main/documentation/JSON-TAB-standard.pdf)
- [JSON-NTV classes and methods](https://loco-philippe.github.io/NTV/json_ntv.html)

This Notebook can also be viewed at [nbviewer](http://nbviewer.org/github/loco-philippe/NTV/tree/main/example)

In [15]:
from json_ntv import Ntv
import pandas as pd
from datetime import date

# the Series Connector is associated with NtvType 'field' in dicobj 
# the DataFrame Connector is associated with NtvType 'tab' in dicobj
option = {'encode_format': 'obj', 'dicobj': {'field': 'SeriesConnec', 'tab': 'DataFrameConnec'}}

## Current Json interface

### Example : simple column
- the interface is not reversible with this example

In [ ]:
df = pd.read_json('{"test integer":[1,2,3], "test string": ["a", "b", "c"]}')
print(df)
# but it is impossible with to_json() to recreate the initial data

### Types and Json interface 

- the only way to keep the types in the json interface is to use the orient='table' option

In [2]:
df  = pd.DataFrame(pd.Series([10,20], name='int32', dtype='Int32'))

# dtype is not included in usual json interface
df.to_json()

'{"int32":{"0":10,"1":20}}'

- only few types are allowed in json-table interface : int64, float64, bool, datetime64, timedelta64, categorical

In [3]:
# 'int32' is lost in json-table interface
df2 = pd.read_json(df.to_json(orient='table'), orient='table')
print(df2.dtypes)
df.equals(df2)

int32    int64
dtype: object


False

- allowed types are not always kept in json-table interface 

In [4]:
df = pd.DataFrame(pd.Series([10,20], name='float64', dtype='float64'))
print(df.dtypes, '\n')
df2 = pd.read_json(df.to_json(orient='records'), orient='records')
print(df2.dtypes)
df.equals(df2)

float64    float64
dtype: object 

float64    int64
dtype: object


False

- data with 'object' dtype is kept only in certain cases

In [20]:
dfd = pd.DataFrame({'dates': [date(2021, 3, 1), date(2021, 3, 3)]})

print(dfd.to_json(default_handler=date.isoformat), '\n')
print(dfd.to_json(orient='table'), '\n')

dfd2 = pd.read_json(dfd.to_json(orient='table'), orient='table')
print(dfd2)

dfd.equals(dfd2)

{"dates":{"0":1614556800000,"1":1614729600000}} 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"dates","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"dates":"2021-03-01T00:00:00.000"},{"index":1,"dates":"2021-03-03T00:00:00.000"}]} 

                     dates
0  2021-03-01T00:00:00.000
1  2021-03-03T00:00:00.000


False

In [6]:
dfd = pd.DataFrame({'tuple': [(2021, 3, 1), (2021, 3, 3)]})
print(dfd, '\n')
print(dfd.to_json(), '\n')
print(dfd.to_json(orient='table'), '\n')
dfd2 = pd.read_json(dfd.to_json(orient='table'), orient='table')
print(dfd2)
dfd.equals(dfd2)

          tuple
0  (2021, 3, 1)
1  (2021, 3, 3) 

{"tuple":{"0":[2021,3,1],"1":[2021,3,3]}} 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"tuple","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"tuple":[2021,3,1]},{"index":1,"tuple":[2021,3,3]}]} 

          tuple
0  [2021, 3, 1]
1  [2021, 3, 3]


False

- with categorical dtype, the underlying dtype is not included in json interface

In [7]:
df = pd.DataFrame(pd.Series([10,20], name='float', dtype='float64'), dtype='category')
print(df.to_json(orient='table'))

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"float","type":"any","constraints":{"enum":[10.0,20.0]},"ordered":false}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"float":10.0},{"index":1,"float":20.0}]}


### Data compactness
- json-table interface is not compact (in this example the size is double or triple the size of the compact format

In [8]:
tab_data = {'dates':           ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'names':           ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          [True, True, True, True, True, True] }

df = pd.DataFrame(tab_data)
print(df, '\n')

# length with compact interface
print(Ntv.obj(df).to_obj(encoded=True))
print(len(Ntv.obj(df).to_obj(encoded=True)), '\n')

# length with actual interface
print(df.to_json(orient='table'))
print(len(df.to_json(orient='table')), '\n')

        dates  value   names  unique
0  1964-01-01     10    john    True
1  1985-02-05     10    eric    True
2  2022-01-21     20  judith    True
3  1964-01-01     20    mila    True
4  1985-02-05     30  hector    True
5  2022-01-21     30   maria    True 

{":tab": {"index": [0, 1, 2, 3, 4, 5], "dates": ["1964-01-01", "1985-02-05", "2022-01-21", "1964-01-01", "1985-02-05", "2022-01-21"], "value": [10, 10, 20, 20, 30, 30], "names": ["john", "eric", "judith", "mila", "hector", "maria"], "unique": [true, true, true, true, true, true]}}
281 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"dates","type":"string"},{"name":"value","type":"integer"},{"name":"names","type":"string"},{"name":"unique","type":"boolean"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"dates":"1964-01-01","value":10,"names":"john","unique":true},{"index":1,"dates":"1985-02-05","value":10,"names":"eric","unique":true},{"index":2,"dates":"2022-01-21","value":20,"names":"judit

In [9]:
tab_data = {'dates':           ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'names':           ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          [True, True, True, True, True, True] }

df = pd.DataFrame(tab_data, dtype='category')
print(df, '\n')

# length with compact interface
print(Ntv.obj(df).to_obj(encoded=True))
print(len(Ntv.obj(df).to_obj(encoded=True)), '\n')

# length with actual interface
print(df.to_json(orient='table'))
print(len(df.to_json(orient='table')), '\n')

        dates value   names unique
0  1964-01-01    10    john   True
1  1985-02-05    10    eric   True
2  2022-01-21    20  judith   True
3  1964-01-01    20    mila   True
4  1985-02-05    30  hector   True
5  2022-01-21    30   maria   True 

{":tab": {"index": [0, 1, 2, 3, 4, 5], "dates": [["1964-01-01", "1985-02-05", "2022-01-21"], [0, 1, 2, 0, 1, 2]], "value": [[10, 20, 30], [0, 0, 1, 1, 2, 2]], "names": [["eric", "hector", "john", "judith", "maria", "mila"], [2, 0, 3, 5, 1, 4]], "unique": [[true], [0, 0, 0, 0, 0, 0]]}}
285 

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"dates","type":"any","constraints":{"enum":["1964-01-01","1985-02-05","2022-01-21"]},"ordered":false},{"name":"value","type":"any","constraints":{"enum":[10,20,30]},"ordered":false},{"name":"names","type":"any","constraints":{"enum":["eric","hector","john","judith","maria","mila"]},"ordered":false},{"name":"unique","type":"boolean","constraints":{"enum":[true]},"ordered":false}],"primaryKey":["i

### Interface is reversible only with json dtype
- see previous examples

### External types
- the interface does not accept external types
- to integrate external types, it is necessary to first create ExtensionArray and ExtensionDtype objects

## Series

### Simple example

In [10]:
field_data = {'value': [1, 2, 3]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype conform to Ntv type
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    1
1    2
2    3
Name: value, dtype: int64

Json representation : 
     {":field": {"value": [1, 2, 3]}}

is Json translation reversible ?  True


### Typed example

In [11]:
field_data = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype conform to Ntv type
print('pandas object :\n', sr)
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
 0   1964-01-01
1   1985-02-05
2   2022-01-21
Name: dates, dtype: datetime64[ns]

Json representation : 
     {":field": {"dates::datetime": ["1964-01-01T00:00:00.000", "1985-02-05T00:00:00.000", "2022-01-21T00:00:00.000"]}}

is Json translation reversible ?  True


### Examples with a non-Pandas type

In [12]:
field_data = {'dates::date': ['1964-01-01', '1985-02-05', '2022-01-21']}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    1964-01-01
1    1985-02-05
2    2022-01-21
Name: dates::date, dtype: object

Json representation : 
     {":field": {"dates::date": ["1964-01-01", "1985-02-05", "2022-01-21"]}}

is Json translation reversible ?  True


In [13]:
field_data = {'coord::point':    [[1,2], [3,4], [5,6]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    POINT (1 2)
1    POINT (3 4)
2    POINT (5 6)
Name: coord::point, dtype: object

Json representation : 
     {":field": {"coord::point": [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]}}

is Json translation reversible ?  True


C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


### Categorical examples
- available only with hashable data

In [14]:
field_data = {"integer": [[1, 2], [0, 1, 0, 1]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    1
1    2
2    1
3    2
Name: integer, dtype: category
Categories (2, int64): [1, 2]

Json representation : 
     {":field": {"integer": [[1, 2], [0, 1, 0, 1]]}}

is Json translation reversible ?  True


In [15]:
field_data = {'dates': [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 0, 2]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    1964-01-01
1    1985-02-05
2    1964-01-01
3    2022-01-21
Name: dates::date, dtype: category
Categories (3, object): [1964-01-01, 1985-02-05, 2022-01-21]

Json representation : 
     {":field": {"dates": [{"::date": ["1964-01-01", "1985-02-05", "2022-01-21"]}, [0, 1, 0, 2]]}}

is Json translation reversible ?  True


In [16]:
field_data = {'array': [{'::array': [[1,2], [3,4], [5,6]]}, [0, 1, 0, 2]]}
field = Ntv.obj({':field': field_data})
sr = field.to_obj(**option)
# pandas dtype : object
print('pandas object :\n' + str(sr))
print('\nJson representation : \n    ', Ntv.obj(sr))
print('\nis Json translation reversible ? ', sr.equals(Ntv.obj(sr).to_obj(**option)))

pandas object :
0    (1, 2)
1    (3, 4)
2    (1, 2)
3    (5, 6)
Name: array::array, dtype: category
Categories (3, object): [(1, 2), (3, 4), (5, 6)]

Json representation : 
     {":field": {"array": [{"::array": [[1, 2], [3, 4], [5, 6]]}, [0, 1, 0, 2]]}}

is Json translation reversible ?  True


## DataFrame

### Initial example

In [17]:
df = pd.DataFrame({"A": list("abca"), "B": list("bccd")})

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', Ntv.obj(df))
print('\nis Json translation reversible ? ', df.equals(Ntv.obj(df).to_obj(**option)))

pandas dtype :
A    object
B    object
dtype: object

pandas object :
   A  B
0  a  b
1  b  c
2  c  c
3  a  d

Json representation : 
     {":tab": {"index": [0, 1, 2, 3], "A": ["a", "b", "c", "a"], "B": ["b", "c", "c", "d"]}}

is Json translation reversible ?  True


### Complete example
- index data
- Pandas dtype (int32, bool, string)
- NTV type (date, point) -> object dtype
- data unique

In [18]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, 30, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             [10, 20, 30, 10, 20, 30],
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], [3,4], [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique':          True }
tab = Ntv.obj({':tab'  : tab_data})
df = tab.to_obj(**option)

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', Ntv.obj(df))
print('\nis Json translation reversible ? ', df.equals(Ntv.obj(df).to_obj(**option)))

C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


pandas dtype :
dates::date     object
value            int64
value32          int32
res              int64
coord::point    object
names           string
unique            bool
dtype: object

pandas object :
    dates::date  value  value32  res coord::point   names  unique
100  1964-01-01     10       12   10  POINT (1 2)    john    True
200  1985-02-05     10       12   20  POINT (3 4)    eric    True
300  2022-01-21     20       22   30  POINT (5 6)  judith    True
400  1964-01-01     20       22   10  POINT (7 8)    mila    True
500  1985-02-05     30       32   20  POINT (3 4)  hector    True
600  2022-01-21     30       32   30  POINT (5 6)   maria    True

Json representation : 
     {":tab": {"index": [100, 200, 300, 400, 500, 600], "dates::date": ["1964-01-01", "1985-02-05", "2022-01-21", "1964-01-01", "1985-02-05", "2022-01-21"], "value": [10, 10, 20, 20, 30, 30], "value32::int32": [12, 12, 22, 22, 32, 32], "res": [10, 20, 30, 10, 20, 30], "coord::point": [[1.0, 2.0], [3.0, 4.0

C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


### Json data can be annotated

In [19]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates::date':     ['1964-01-01', '1985-02-05', '2022-01-21', '1964-01-01', '1985-02-05', '2022-01-21'], 
            'value':           [10, 10, 20, 20, {'valid?': 30}, 30],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique:boolean':  True }
tab = Ntv.obj({':tab'  : tab_data})
df2 = tab.to_obj(**option)

print('is DataFrame identical ? ', df.equals(df2))

is DataFrame identical ?  True


C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


### Categorical data can be included

In [20]:
df = pd.DataFrame({"A": list("abca"), "B": list("bccd")}, dtype="category")

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', Ntv.obj(df))
print('\nis Json translation reversible ? ', df.equals(Ntv.obj(df).to_obj(**option)))

pandas dtype :
A    category
B    category
dtype: object

pandas object :
   A  B
0  a  b
1  b  c
2  c  c
3  a  d

Json representation : 
     {":tab": {"index": [0, 1, 2, 3], "A": [["a", "b", "c"], [0, 1, 2, 0]], "B": [["b", "c", "d"], [0, 1, 1, 2]]}}

is Json translation reversible ?  True


In [21]:
tab_data = {'index':           [100, 200, 300, 400, 500, 600],
            'dates':           [{'::date': ['1964-01-01', '1985-02-05', '2022-01-21']}, [0, 1, 2, 0, 1, 2]],
            'value':           [[10, 20, {'valid?': 30}], [0, 0, 1, 1, 2, 2]],
            'value32::int32':  [12, 12, 22, 22, 32, 32],
            'res':             {'res1': 10, 'res2': 20, 'res3': 30, 'res4': 10, 'res5': 20, 'res6': 30},
            'coord::point':    [[1,2], [3,4], [5,6], [7,8], {'same as 2nd point': [3,4]}, [5,6]],
            'names::string':   ['john', 'eric', 'judith', 'mila', 'hector', 'maria'],
            'unique:boolean':  True }
tab = Ntv.obj({':tab'  : tab_data})
df = tab.to_obj(**option)

print('pandas dtype :\n' + str(df.dtypes))
print('\npandas object :\n' + str(df))
print('\nJson representation : \n    ', Ntv.obj(df))
print('\nis Json translation reversible ? ', df.equals(Ntv.obj(df).to_obj(**option)))

pandas dtype :
dates::date     category
value           category
value32            int32
res                int64
coord::point      object
names             string
unique              bool
dtype: object

pandas object :
    dates::date value  value32  res coord::point   names  unique
100  1964-01-01    10       12   10  POINT (1 2)    john    True
200  1985-02-05    10       12   20  POINT (3 4)    eric    True
300  2022-01-21    20       22   30  POINT (5 6)  judith    True
400  1964-01-01    20       22   10  POINT (7 8)    mila    True
500  1985-02-05    30       32   20  POINT (3 4)  hector    True
600  2022-01-21    30       32   30  POINT (5 6)   maria    True

Json representation : 
     {":tab": {"index": [100, 200, 300, 400, 500, 600], "dates": [{"::date": ["1964-01-01", "1985-02-05", "2022-01-21"]}, [0, 1, 2, 0, 1, 2]], "value": [[10, 20, 30], [0, 0, 1, 1, 2, 2]], "value32::int32": [12, 12, 22, 22, 32, 32], "res": [10, 20, 30, 10, 20, 30], "coord::point": [[1.0, 2.0], [3.0, 

C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)
C:\Users\phili\anaconda3\envs\observ_v10_20230309\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)
